<a href="https://colab.research.google.com/github/rath772k/Machine-learning-problems/blob/master/AnimalPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/DL%23+Beginner.zip

--2019-03-28 06:59:18--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/DL%23+Beginner.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.36.29
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.36.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6873530111 (6.4G) [application/zip]
Saving to: ‘DL#+Beginner.zip’

DL#+Beginner.zip    100%[===================>]   6.40G  19.9MB/s    in 5m 47s  

2019-03-28 07:05:06 (18.9 MB/s) - ‘DL#+Beginner.zip’ saved [6873530111/6873530111]



In [0]:
import shutil
shutil.move("DL#+Beginner.zip","gdrive/My Drive")

'gdrive/My Drive/DL#+Beginner.zip'

In [0]:
!7za x 'gdrive/My Drive/DL#+Beginner.zip'


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 6873530111 bytes (6556 MiB)

Extracting archive: gdrive/My Drive/DL#+Beginner.zip

ERRORS:
Headers Error

--
Path = gdrive/My Drive/DL#+Beginner.zip
Type = zip
ERRORS:
Headers Error
Physical Size = 6873530111
64-bit = +

  0%      0% 1 - DL# Beginner/test.zip                                1% 1 - DL# Beginner/test.zip                                2% 1 - DL# Beginner/test.zip                                3% 1 - DL# Beginner/test.zip                                4%

In [0]:
!7za x 'DL# Beginner/meta-data.zip'


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1692896 bytes (1654 KiB)

Extracting archive: DL# Beginner/meta-data.zip
--
Path = DL# Beginner/meta-data.zip
Type = zip
Physical Size = 1692896

  0%    Everything is Ok

Files: 3
Size:       3917481
Compressed: 1692896


In [0]:
shutil.move("test","gdrive/My Drive")
shutil.move("train","gdrive/My Drive")
shutil.move("meta-data","gdrive/My Drive")

'gdrive/My Drive/meta-data'

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
from keras.layers import *
from keras.models import Model
from sklearn.utils import class_weight
import os
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam,SGD

Using TensorFlow backend.


In [0]:
train=pd.read_csv("gdrive/My Drive/meta-data/train.csv")

In [0]:
train.groupby('Animal').count()

,Image_id
Animal,
antelope,695
bat,256
beaver,133
bobcat,418
buffalo,606
chihuahua,386
chimpanzee,477
collie,680
dalmatian,359


In [0]:
list(train.Image_id.values)

In [0]:
import os
file_names=list(train['Image_id'].values)
labels=list(train['Animal'].values)
uni_labels=np.unique(labels)

In [0]:
import random

In [0]:
#for name in uni_labels:
 # os.makedirs("gdrive/My Drive/validation/"+name)
for i in random.sample(range(len(labels)),len(labels)//10*2):
  shutil.move("gdrive/My Drive/train/"+labels[i]+"/"+file_names[i],"gdrive/My Drive/validation/"+labels[i])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
from keras.applications import inception_resnet_v2

In [5]:
model=inception_resnet_v2.InceptionResNetV2(include_top=False,input_shape=(299,299,3))


Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 3s 0us/step


In [0]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(2048,activation='relu')(x) #we add dense layers so that the model can 
#learn more complex functions and classify for better results.
X=BatchNormalization(axis=-1)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
X=BatchNormalization(axis=-1)
x=Dense(1024,activation='relu')(x) #dense layer 3
X=BatchNormalization(axis=-1)
x=Dense(512,activation='relu')(x) #dense layer 4
X=BatchNormalization(axis=-1)(x)
preds=Dense(30,activation='softmax')(x) #final layer with softmax activation

In [0]:
model1=Model(inputs=model.input,outputs=preds)
#if we want to set the cnn layers of the network to be non-trainable
for layer in model.layers:
    layer.trainable=False

In [0]:
sgd = SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    brightness_range=(0.6,1),
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    'gdrive/My Drive/train',
    target_size=(299,299),
    batch_size=64,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'gdrive/My Drive/validation',
    target_size=(299,299),
    batch_size=64,
    class_mode='categorical')

Found 10400 images belonging to 30 classes.
Found 2600 images belonging to 30 classes.


Giving class weights

In [0]:
files_per_class = []
for folder in os.listdir("gdrive/My Drive/train"):
    if not os.path.isfile(folder):
            files_per_class.append(len(os.listdir("gdrive/My Drive/train" 
                                                  + '/' + folder)))
total_files = sum(files_per_class)
class_weights = {}
for i in range(len(files_per_class)):
    class_weights[i] = 1 - (float(files_per_class[i]) / total_files)

In [0]:
filepath="gdrive/My Drive/weightsMLBegChalHackerearth"
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.3,
                                    cooldown=0, patience=5, min_lr=1e-5)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
model1.fit_generator(generator=train_generator,
                   steps_per_epoch=np.ceil(10400/64),
                   epochs=10,
                   validation_data=validation_generator,
    validation_steps=np.ceil(2600/64),class_weight=class_weights,callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
163/163 [==============================] - 3661s 22s/step - loss: 0.2102 - acc: 0.9325 - val_loss: 0.2822 - val_acc: 0.9277

Epoch 00001: val_acc improved from -inf to 0.92769, saving model to gdrive/My Drive/weightsMLBegChalHackerearth
Epoch 2/10
163/163 [==============================] - 595s 4s/step - loss: 0.2144 - acc: 0.9326 - val_loss: 0.2822 - val_acc: 0.9277

Epoch 00002: val_acc improved from 0.92769 to 0.92769, saving model to gdrive/My Drive/weightsMLBegChalHackerearth
Epoch 3/10
163/163 [==============================] - 604s 4s/step - loss: 0.2191 - acc: 0.9318 - val_loss: 0.2822 - val_acc: 0.9277

Epoch 00003: val_acc did not improve from 0.92769
Epoch 4/10
163/163 [==============================] - 603s 4s/step - loss: 0.2133 - acc: 0.9339 - val_loss: 0.2822 - val_acc: 0.9277

Epoch 00004: val_acc did not improve from 0.92769
Epoch 5/10
163/163 [==============================] - 611s 4s/step - loss: 0.2225 - acc

In [0]:
model1.load_weights("gdrive/My Drive/weightsMLBegChalHackerearth")

In [0]:
import glob
filename=glob.glob('gdrive/My Drive/test/*.jpg')
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

filename.sort(key=natural_keys)

In [0]:
test=np.array([cv2.resize(cv2.imread(file),(299,299))/255 for file in filename[0:60]])

In [0]:
result=model1.predict(test)

In [0]:
for i in range(1,100):
  test=np.array([cv2.resize(cv2.imread(file),(299,299))/255 for file in filename[i*60:i*60+60]])
  result=np.append(result,model1.predict(test),axis=0)

In [0]:
result.shape

(6000, 30)

In [0]:
sub=pd.read_csv("gdrive/My Drive/meta-data/sample_submission.csv")

In [0]:
result[0]

array([2.33367564e-06, 4.78389026e-08, 1.97519636e-08, 1.89859531e-06,
       9.96147156e-01, 1.26690821e-07, 2.11221135e-07, 4.20305014e-07,
       1.06108125e-07, 1.15009857e-06, 2.88552750e-04, 4.48722094e-05,
       3.72888462e-05, 3.15077120e-09, 4.70893946e-09, 1.09508587e-03,
       6.59943353e-08, 1.30232237e-07, 2.04031030e-03, 4.47302313e-07,
       9.14945304e-07, 5.72541445e-08, 3.13807366e-04, 1.78810228e-06,
       2.04078688e-07, 4.22134816e-09, 1.40943882e-07, 2.18885834e-05,
       7.51333346e-07, 3.52071339e-07], dtype=float32)

In [0]:
sub.iloc[:,1:]=result

In [0]:
sub.to_csv('submission.csv',index=False,float_format='%.20f')

In [0]:
from google.colab import files

In [0]:
files.download('gdrive/My Drive/submission.csv')

In [0]:
shutil.move("submission.csv","gdrive/My Drive")

'gdrive/My Drive/submission.csv'